In [1]:
import ctypes

def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [2]:
class Person:
    def __init__(self, name):
        self.name = name
    
    def __repr__(self):
        return f'Person({self.name})'
    
    def __del__(self):
        print(f'__del__ called for {self}...')

In [3]:
p = Person('Alex')

In [4]:
id_p = id(p)
ref_count(id_p)

1

In [5]:
p = None

__del__ called for Person(Alex)...


In [6]:
p = Person('Alex')
del p

__del__ called for Person(Alex)...


In [7]:
class Person:
    def __init__(self, name):
        self.name = name
    
    def __repr__(self):
        return f'Person({self.name})'
    
    def __del__(self):
        print(f'__del__ called for {self}...')

    def gen_ex(self):
        raise ValueError('Something went bump...')

In [8]:
p = Person('Eric')
p_id = id(p)
ref_count(p_id)

1

In [11]:
try:
    p.gen_ex()
except ValueError as ex:
    error = ex
    print(ex)

Something went bump...


In [12]:
error

ValueError('Something went bump...')

In [13]:
ref_count(p_id)

2

In [14]:
type(error)

ValueError

In [15]:
dir(error)

['__cause__',
 '__class__',
 '__context__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__suppress_context__',
 '__traceback__',
 'add_note',
 'args',
 'with_traceback']

In [16]:
dir(error.__traceback__)

['tb_frame', 'tb_lasti', 'tb_lineno', 'tb_next']

In [17]:
dir(error.__traceback__.tb_frame)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'f_back',
 'f_builtins',
 'f_code',
 'f_globals',
 'f_lasti',
 'f_lineno',
 'f_locals',
 'f_trace',
 'f_trace_lines',
 'f_trace_opcodes']

In [19]:
type(error.__traceback__.tb_frame.f_locals)

dict

In [23]:
for key, value in error.__traceback__.tb_frame.f_locals.copy().items():
    if isinstance(value, Person):
        print(key, value, id(value))

p Person(Eric) 1757004142912


In [24]:
ref_count(p_id)

2

In [25]:
del p

In [26]:
del error

In [27]:
ref_count(p_id)

1

In [28]:
class Person:
    def __del__(self):
        raise ValueError('Something went bump...')

In [29]:
p = Person()

In [30]:
del p
print('All good, no exceptions interupted our code.')

Exception ignored in: <function Person.__del__ at 0x00000199160A0FE0>
Traceback (most recent call last):
  File "C:\Users\CASPER\AppData\Local\Temp\ipykernel_34528\795576010.py", line 3, in __del__
ValueError: Something went bump...


All good, no exceptions interupted our code.


In [31]:
import sys

In [32]:
sys.stderr, sys.stdout

(<ipykernel.iostream.OutStream at 0x199144c5f30>,
 <ipykernel.iostream.OutStream at 0x19914507340>)

In [33]:
class ErrToFile:
    def __init__(self, fname):
        self._fname = fname
        self._current_stderr = sys.stderr

    def __enter__(self):
        self._file = open(self._fname, 'w')
        sys.stderr = self._file

    def __exit__(self, exc_type, exc_value, exc_tb):
        sys.stderr = self._current_stderr
        if self._file:
            self._file.close()
        return False

In [34]:
p = Person()

In [35]:
with ErrToFile('err.txt'):
    del p
    print(100)
print(200)
print(300)

100
200
300


In [36]:
with open('err.txt') as f:
    print(f.readlines())

['Exception ignored in: <function Person.__del__ at 0x00000199160A0FE0>\n', 'Traceback (most recent call last):\n', '  File "C:\\Users\\CASPER\\AppData\\Local\\Temp\\ipykernel_34528\\795576010.py", line 3, in __del__\n', 'ValueError: Something went bump...\n']
